<a target="_blank" href="https://colab.research.google.com/github/ArtificialIntelligenceToolkit/aitk/blob/master/notebooks/HillClimbing.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install aitk --upgrade --quiet

<a href="https://colab.research.google.com/github/lmeeden1/aitk.networks/blob/master/HillClimbing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Hill Climbing to find food

This notebook demonstrates how a robot can use a "smell" sensor to to locate and eat food in an open world by using a simple hill climbing algorithm to follow the gradient of the order towards the food.

In [ ]:
%pip install aitk --quiet

In [ ]:
from aitk.robots import World, Scribbler, SmellSensor

We will place food at the center of this world.

In [ ]:
world = World(width=200, height=200, scale=2, boundary_wall=False)
world.add_food(100, 100, 70)

Random seed set to: 5683051


We equip the robot with a smell sensor on its found center.

In [ ]:
robot = Scribbler(x=110, y=150, a=50)
robot.add_device(SmellSensor(position=(8,0),name="nose"))
robot.set_max_trace_length(60)
world.add_robot(robot)
world.update()

In [ ]:
from random import random

We need to set some intial parameters for the robot's controller.

In [ ]:
def init_state(robot):
    robot.state['behavior'] = 'test'
    robot.state['max_step'] = 5
    robot.state['current'] = 0
    robot.state['step'] = 0
    robot.state['rotation'] = 0

We will use a finite state machine style controller that is based on a limited set of behaviors:

* **test** is used to sniff for food and then initiate a turn to find a gradient or difference in smell
* **turn** rotates a random direction and compares the new smell amount and decides whether to go forward, if the smell is increasing, or whether to reverse the last move if the smell is decresing
* **reverse** undoes the previous robot move
* **forward** moves the robot forward

To summarizem, the robot always begins in **test** mode and then goes to **turn**. From there it either goes to **forward** or **reverse**, and then back to **test** mode.

In [ ]:
def controller(robot):
    if robot.eat():
        # Try to eat, if successful end run
        robot.speak("FOUND FOOD!")
        return True
    elif robot.state['behavior'] == 'test':
        # Check current smell and choose a random rotation to try
        robot.speak("test")
        robot.state['current'] = robot['nose'].get_reading()
        robot.state['rotation'] = random()-0.5
        robot.move(0, robot.state['rotation'])
        robot.state['behavior'] = 'turn'
    elif robot.state['behavior'] == 'turn':
        # Turn for max_steps, then check new smell
        robot.speak("turn")
        if robot.state['step'] % robot.state['max_step'] == 0:
            next_reading = robot['nose'].get_reading()
            delta = next_reading - robot.state['current']
            if delta >= 0:
                # if smell is stronger or the same, go forward
                robot.move(0.5, 0)
                robot.state['behavior'] = 'forward'
            else:
                # if smell is weaker, reverse turn and try again
                robot.move(0, -1*robot.state['rotation'])
                robot.state['behavior'] = 'reverse'
    elif robot.state['behavior'] == 'reverse':
        # Reverse rotation for max_steps, then go back to testing
        robot.speak("reverse")
        if robot.state['step'] % robot.state['max_step'] == 0:
            robot.stop()
            robot.state['behavior'] = 'test'
    elif robot.state['behavior'] == 'forward':
        # Move forward for max_steps, then go back to testing
        robot.speak("forward")
        if robot.state['step'] % robot.state['max_step'] == 0:
            robot.stop()
            robot.state['behavior'] = 'test'
    robot.state['step'] += 1

Let's set up a way to watch the robot's behavior.

In [ ]:
world.watch()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

Reset the world, place the robot in a random location, and start off the controller to find the food.

In [ ]:
world.reset()
robot.set_random_pose()
init_state(robot)
world.run([controller],real_time=True)

0it [00:00, ?it/s]

Simulation stopped at: 00:00:16.20; speed 0.98 x real time


True